In [1]:
from collections import defaultdict
import tensorflow as tf
import numpy as np
import tflearn
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import tifffile as tiff
import sys
import sklearn
import time
import PIL
from scipy import misc
import image_tool

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X_train = np.load('my_data/new_data.npy', 'r')
y_train = np.load('my_data/new_label.npy', 'r')
y_train = 1. - y_train

In [3]:
X_train.shape, y_train.shape

((2650, 112, 112, 8), (2650, 112, 112, 1))

In [2]:
def bn_relu_conv_concat(bottom, training, name, filters=12, stride=(1, 1), activation=tf.nn.relu):
    with tf.variable_scope("dense_layer_{}".format(name)):
        net = tf.layers.batch_normalization(bottom, training=training, name='bn_dense_{}'.format(name))
        net = activation(net, name='relu_dense_{}'.format(name))
        net = tf.layers.conv2d(net, filters, (3, 3), strides=stride, activation=None, padding='same', name='conv_dense_{}'.format(name))
        net = tf.concat([net, bottom], axis=3, name='concat_dense_{}'.format(name))
        return net

def transition(bottom, training, name, stride=(1, 1), activation=tf.nn.relu, pool=True):
    with tf.variable_scope("transition_layer_{}".format(name)):
        filters = bottom.get_shape().as_list()[3]
        net = tf.layers.batch_normalization(bottom, training=training, name='bn_transition_{}'.format(name))
        base = activation(net, name='relu_transition_{}'.format(name))
        net = tf.layers.conv2d(base, filters, (1, 1), strides=stride, activation=None, padding='same', name='conv_transition_{}'.format(name))
        if pool:
            net = tf.layers.average_pooling2d(net, (2, 2), strides=(2, 2), name="pool_{}".format(name))
        return base, net

def upsampling_2D(tensor, name, size=(2, 2)):
    H, W, _ = tensor.get_shape().as_list()[1:]

    H_multi, W_multi = size
    target_H = H * H_multi
    target_W = W * W_multi

    return tf.image.resize_nearest_neighbor(tensor, (target_H, target_W), name="upsample_{}".format(name))

def up_sample_conv(net, filters, strides, training, name, activation=tf.nn.relu):
    with tf.variable_scope("up_sample_{}".format(name)):
        net = tf.layers.conv2d(net, filters, (1, 1), activation=activation, padding="same", name="up_conv")
        if strides != 1:
            net = upsampling_2D(net, name, size=(strides, strides))
        with tf.variable_scope("conv_1"):
            conv_1 = tf.layers.conv2d(net, 1, (3, 3), activation=None, padding="same", name="conv")
            net = tf.layers.batch_normalization(conv_1, training=training, name="bn")
            net = activation(net, name="relu")
        with tf.variable_scope("conv_2"):
            conv_1 = tf.layers.conv2d(net, 1, (3, 3), activation=None, padding="same", name="conv")
            net = tf.layers.batch_normalization(conv_1, training=training, name="bn")
            net = activation(net, name="relu")
        net = tf.layers.conv2d(net, 1,  [1, 1], activation=None, name="Final")
        return net

def dense_net(X, training):
    with tf.variable_scope("Preprocessing"):
        net = X * 2 - 1
        net = tf.layers.conv2d(net, 8, (1, 1), name="color_space1_adjust")
    dsn1 = tf.layers.conv2d(net, 16, (7, 7), padding='same', name='first_conv', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(dsn1, 3, strides=2, padding='same', name='first_pool')
    
    with tf.variable_scope("dense_block_1"):
        for i in range(0, 6):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn2, net = transition(net, training, 101)
    
    with tf.variable_scope("dense_block_2"):
        for i in range(8, 20):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn3, net = transition(net, training, 102)
    
    with tf.variable_scope("dense_block_3"):
        for i in range(20, 48):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn4, net = transition(net, training, 103)
    
    with tf.variable_scope("dense_block_4"):
        for i in range(48, 64):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn5, net = transition(net, training, 104)
    
    return dsn1, dsn2, dsn3, dsn4, dsn5
    
def make_hed(X, training):
    dsn1, dsn2, dsn3, dsn4, dsn5 = dense_net(X, training)
    
    dsn1 = up_sample_conv(dsn1, 16, 1, training, "1")
    dsn2 = up_sample_conv(dsn2, 32, 2, training, "2")
    dsn3 = up_sample_conv(dsn3, 64, 4, training, "3")
    dsn4 = up_sample_conv(dsn4, 128, 8, training, "4")
    dsn5 = up_sample_conv(dsn5, 256, 16, training, "5")
    
    dsn = tf.concat([dsn1, dsn2, dsn3, dsn4, dsn5], axis=3, name='concat')
    dsn = tf.reshape(dsn, [-1, 112, 112, 5])
    dsn = tf.layers.conv2d(dsn, 1, (1, 1), name='output', activation=None, padding='same')

    return dsn1, dsn2, dsn3, dsn4, dsn5, dsn

def IOU_loss(y_pred, y_true):
    y_pred = tf.round(1. - y_pred)
    y_true = tf.round(1. - y_true)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    
    H, W, _ = y_pred.get_shape().as_list()[1:]
    pred_flat = tf.reshape(y_pred, [-1, H * W])
    true_flat = tf.reshape(y_true, [-1, H * W])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + 1e-12
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + 1e-7

    return tf.reduce_mean(intersection / denominator)

def IOU_no_classified(y_pred, y_true):
    y_pred = 1. - y_pred
    y_true = 1. - y_true
    
    H, W, _ = y_pred.get_shape().as_list()[1:]
    pred_flat = tf.reshape(y_pred, [-1, H * W])
    true_flat = tf.reshape(y_true, [-1, H * W])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + 1e-12
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + 1e-7

    return tf.reduce_mean(intersection / denominator)

def class_balanced_sigmoid_cross_entropy(y_true, y_pred):
    y = tf.cast(y_true, tf.float32)
    
    count_neg = tf.reduce_sum(1. - y) + 1
    count_pos = tf.reduce_sum(y) + 1
    beta = count_neg / (count_neg + count_pos + 2)
    
    pos_weight = beta / (1 - beta)
    cost = tf.nn.weighted_cross_entropy_with_logits(y, y_pred, pos_weight)
    cost = tf.reduce_mean(cost * (1 - beta))
    
    return cost

In [3]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True
sess = tf.Session(config=config)
with tf.device("/gpu:7"):
    global_step_tensor = tf.Variable(1, trainable=False, name='global_step')
    with tf.variable_scope("X_Input_Layer"):
        X_layer = tf.placeholder(shape=[None, 112, 112, 8], dtype=tf.float32, name="X_layer")
    with tf.variable_scope("Y_Input_Layer"):
        y_layer = tf.placeholder(shape=[None, 112, 112, 1], dtype=tf.float32, name="y_layer")
    with tf.variable_scope("Mode_Layer"):
        trainning_mode = tf.placeholder(tf.bool, name="Mode")
    d1, d2, d3, d4, d5, pred_not_sigmoid = make_hed(X_layer, trainning_mode)
    pred = tf.nn.sigmoid(pred_not_sigmoid)
    tf.summary.image("Predicted_Mask", pred)
    X_layer_2015 = X_layer[:, :, :, :3]
    X_layer_2017 = X_layer[:, :, :, 4:7]
    tf.summary.image("2015", X_layer_2015)
    tf.summary.image("2017", X_layer_2017)
    tf.summary.image("Label", y_layer)
    
    with tf.variable_scope("Accuracy"):
        pred_int = tf.cast(tf.round(pred, name="round_pred"), dtype=tf.int32, name="cast_pred")
        y_int = tf.cast(tf.round(y_layer, name="round_y"), dtype=tf.int32, name="cast_y")
        acc = tf.contrib.metrics.accuracy(y_int, pred_int)
        tf.summary.scalar("Accuracy", acc)
        
    with tf.variable_scope("Loss"):
        loss = class_balanced_sigmoid_cross_entropy(y_layer, pred_not_sigmoid)
        tf.summary.scalar("Loss", loss)
    
    with tf.variable_scope("IOU"):
        IOU_op = IOU_loss(pred, y_layer)
        tf.summary.scalar("IOU", IOU_op)
        
    with tf.variable_scope("IOU_no_classified"):
        IOU_op_no_class = IOU_no_classified(pred, y_layer)
        tf.summary.scalar("IOU_no_classified", IOU_op_no_class)
        
    summary_op = tf.summary.merge_all()
    
    with tf.variable_scope("Test_Summary_INPUT"):
        test_IOU_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_IOU")
        test_IOU_no_class_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_IOU_NO_CLASSIFY")
        test_acc_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_Acc")
        test_loss_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_Loss")
    test_summary_IOU_op = tf.summary.scalar("epoch/IOU", test_IOU_input)
    test_summary_IOU_op_no_classified = tf.summary.scalar("epoch/IOU_no_classified", test_IOU_no_class_input)
    test_summary_acc_op = tf.summary.scalar("epoch/Accuracy", test_acc_input)
    test_summary_loss_op = tf.summary.scalar("epoch/Loss", test_loss_input)

    lr = tf.placeholder(tf.float32, name='Learning_Rate')
    choose_momentum = tf.placeholder(tf.int32, name='Choose_Optimizer')
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        global_step = tf.train.get_global_step()
        if choose_momentum == 1:
            train_step = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(loss, global_step=global_step)
        else:
            train_step = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)

In [6]:
!rm -rf tmp/Dense_HED/log_train
!rm -rf tmp/Dense_HED/log_test

In [7]:
train_summary_writer = tf.summary.FileWriter("tmp/Dense_HED/log_train", sess.graph)
test_summary_writer  = tf.summary.FileWriter("tmp/Dense_HED/log_test", sess.graph)

In [4]:
# sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, "../Step_1_Prediction/my_model/Dense_HED/-121")
tf.train.global_step(sess, global_step_tensor)

INFO:tensorflow:Restoring parameters from ../Step_1_Prediction/my_model/Dense_HED/-121


183977

In [5]:
def random_augmentation(imges, labels):
    au_images = []
    au_labels = []
    for i in range(len(imges)):
        img_tmp = np.copy(imges[i])
        label_tmp = np.copy(labels[i])
        rand_flip = np.random.randint(0, 4)
        rand_rotat = np.random.randint(0, 4)
        rand_change = np.random.randint(0, 2)
        if rand_flip != 3:
            img_tmp = cv2.flip(img_tmp, rand_flip - 1)
            label_tmp = cv2.flip(label_tmp, rand_flip - 1)
        M = cv2.getRotationMatrix2D((56, 56), rand_rotat*90, 1.0)
        img_tmp = cv2.warpAffine(img_tmp, M, (112, 112))
        label_tmp = cv2.warpAffine(label_tmp, M, (112, 112))
        if rand_change == 1:
            mean1 = np.mean(img_tmp[:, :, :4])
            mean2 = np.mean(img_tmp[:, :, 4:])
            rand_std1 = np.random.ranf() * 0.4 + 0.8
            rand_std2 = np.random.ranf() * 0.4 + 0.8
            rand_mean1 = (np.random.ranf() * 0.4 - 0.2) * mean1
            rand_mean2 = (np.random.ranf() * 0.4 - 0.2) * mean2
            img_tmp[:, :, :4] = (img_tmp[:, :, :4] * rand_std1 + rand_mean1).clip(0, 1)
            img_tmp[:, :, 4:] = (img_tmp[:, :, 4:] * rand_std2 + rand_mean2).clip(0, 1)
        au_images.append(img_tmp)
        au_labels.append(label_tmp)
    au_images = np.asarray(au_images)
    au_labels = np.asarray(au_labels).reshape(-1, 112, 112, 1)
    return au_images, au_labels

In [16]:
batch_size = 32
start_time = time.time()
learning_rate = 0.0001
change_step = 16
momentum_optimizer = 1
adam_step = 0

for i in range(0, 300):
    train_IOU = 0.0
    train_IOU_no_classified = 0.0
    train_accuary = 0.0
    train_loss = 0.0
    rand_indexes = np.random.choice(len(X_train), len(X_train), replace = False)
    for j in range(0, len(X_train), batch_size):
        X_batch, y_batch = random_augmentation(X_train[rand_indexes[j: j + batch_size]], y_train[rand_indexes[j: j + batch_size]])
        _, step_iou, step_iou_no_classified, step_acc, step_loss, step_summary, step_value = sess.run(
            [train_step, IOU_op, IOU_op_no_class, acc, loss, summary_op, global_step],
            feed_dict = {X_layer: X_batch, y_layer: y_batch, trainning_mode: True,\
                         lr: learning_rate, choose_momentum: momentum_optimizer})
        train_IOU += step_iou * X_batch.shape[0]
        train_IOU_no_classified += step_iou_no_classified * X_batch.shape[0]
        train_accuary += step_acc * X_batch.shape[0]
        train_loss += step_loss * X_batch.shape[0]
        train_summary_writer.add_summary(step_summary, step_value)

        percents = 100 * j / len(X_train)
        line_str = "Training: [{0}>{1}], {2}%, {3}/{4}, IOU:{5:.2f}, IOU_no:{6:.2f}, Accuracy:{7:.2f}, Loss:{8:.2f}, {9}s".format(
            "=" * (int(percents) // 2), " " * (50 - int(percents) // 2),
            int(percents), j, len(X_train), step_iou, step_iou_no_classified, step_acc, step_loss,
            int(time.time() - start_time))
        sys.stdout.write("\r" + line_str)
    train_IOU /= (len(X_train))
    train_IOU_no_classified /= (len(X_train))
    train_accuary /= (len(X_train))
#     train_loss /= 16
    
    test_IOU_summary = sess.run(test_summary_IOU_op, feed_dict={test_IOU_input: train_IOU})
    train_summary_writer.add_summary(test_IOU_summary, i)
    
    test_IOU_no_classified_summary = sess.run(test_summary_IOU_op_no_classified, feed_dict={test_IOU_no_class_input: train_IOU_no_classified})
    train_summary_writer.add_summary(test_IOU_no_classified_summary, i)
    
    test_acc_summary = sess.run(test_summary_acc_op, feed_dict={test_acc_input: train_accuary})
    train_summary_writer.add_summary(test_acc_summary, i)
    
    test_loss_summary = sess.run(test_summary_loss_op, feed_dict={test_loss_input: train_loss})
    train_summary_writer.add_summary(test_loss_summary, i)
    
    print()
    print("EPOCH {0}, IOU: {1:.2f}, IOU_no:{2:.2f}, Accuracy: {3:.2f}, Loss: {4:.2f}".format(i, train_IOU, train_IOU_no_classified, train_accuary, train_loss))
    if i % 1 == 0:
        saver = tf.train.Saver()
        saver.save(sess, 'my_model/Dense_HED/', global_step=i)

Training: [=================================================> ], 99%, 2624/2650, IOU:0.78, IOU_no:0.71, Accuracy:0.94, Loss:0.04, 157s
EPOCH 0, IOU: 0.75, IOU_no:0.67, Accuracy: 0.93, Loss: 103.11
Training: [=================================================> ], 99%, 2624/2650, IOU:0.63, IOU_no:0.56, Accuracy:0.92, Loss:0.03, 395s
EPOCH 1, IOU: 0.73, IOU_no:0.66, Accuracy: 0.93, Loss: 103.02
Training: [=================================================> ], 99%, 2624/2650, IOU:0.79, IOU_no:0.72, Accuracy:0.94, Loss:0.04, 678s
EPOCH 2, IOU: 0.75, IOU_no:0.67, Accuracy: 0.93, Loss: 104.14
Training: [=================================================> ], 99%, 2624/2650, IOU:0.84, IOU_no:0.77, Accuracy:0.94, Loss:0.04, 946s
EPOCH 3, IOU: 0.75, IOU_no:0.68, Accuracy: 0.93, Loss: 103.52
Training: [=================================================> ], 99%, 2624/2650, IOU:0.69, IOU_no:0.61, Accuracy:0.94, Loss:0.04, 1156s
EPOCH 4, IOU: 0.74, IOU_no:0.67, Accuracy: 0.93, Loss: 102.80
Training: [===

KeyboardInterrupt: 

In [6]:
saver = tf.train.Saver()
saver.restore(sess, "my_model/Dense_HED/-4")

INFO:tensorflow:Restoring parameters from my_model/Dense_HED/-4


In [5]:
# new_data = np.load('../new_data/data.npy', 'r')
new_data = np.load('../new_data/new_data.npy', 'r')
new_data.shape

(3000, 15106, 8)

In [6]:
# ensemble
pre_label = np.load('../Step_2.2_Rerank/results/remove_small.npy', 'r')
pre_label = np.copy(pre_label.reshape(3000, 15106, 1))

In [7]:
pre_label.shape, new_data.shape

((3000, 15106, 1), (3000, 15106, 8))

In [8]:
pre_label = image_tool.mask_remove_small(pre_label, 400)

[=================================================>], 497/498, 99%, 0.03s 

In [9]:
new_rects, new_areas, new_contours = image_tool.mask_to_contours_index(np.copy(pre_label))
new_rects.shape

(498, 4)

In [10]:
new_label = np.asarray(pre_label, dtype=np.float32).reshape(3000, 15106)
tmp_label = np.asarray(pre_label, dtype=np.float32).reshape(3000, 15106)
pre_label = pre_label.reshape(3000, 15106)

In [11]:
from utils import ProgressBar
pb = ProgressBar(worksum=len(new_rects))
pb.startjob()
for i in range(len(new_rects)):
    left, top, right, bottom = new_rects[i]
    
    width = right - left
    height = bottom - top
    
    left = max(left - width//4, 0)
    top = max(top - height//4, 0)
    right = min(right + width//4, 15106)
    bottom = min(bottom + width//4, 3000)
    
    resize_width = (width // 112 + 1) * 112
    resize_height = (height // 112 + 1) * 112
    
    one_result = np.zeros((resize_height, resize_width))
    
    one_X = cv2.resize(new_data[top:bottom, left:right, :], (resize_width, resize_height))
    one_y = pre_label[top:bottom, left:right]
    
    for i in range(0, resize_width, 112):
        for j in range(0, resize_height, 112):
            one_label = sess.run(pred, feed_dict={X_layer: one_X[j:j+112, i:i+112, :].reshape(1, 112, 112, 8), trainning_mode:False})
            one_result[j:j+112, i:i+112] = one_label.reshape(112, 112)
    
    one_result = np.asarray(cv2.resize(one_result, (right - left, bottom - top)), dtype=np.float32)
    one_result[one_y == 0] = 0
    tmp_label[top:bottom, left:right] = 1. - one_result
        
    pb.complete(1)

 100.00 % [==================================================>] 498/498 	 used:32s eta:0 s

In [12]:
tmp_label.shape, tmp_label.dtype

((3000, 15106), dtype('float32'))

In [13]:
np.save('result/Dense', tmp_label)

In [14]:
HED = np.load('result/HED.npy', 'r')
Res = np.load('result/Res.npy', 'r')
Dense = np.load('result/Dense.npy', 'r')
Ensemble = HED + Res + Dense
Ensemble /= 3
Ensemble = np.asarray(Ensemble >= 0.5, dtype=np.uint8)

In [15]:
def mask_replace_non_exist(pre_mask, new_mask, rects, contours, areas, replace_threshold=0.5, delete_threshold=0.2):
    mask = np.copy(pre_mask)
    for index, rect in enumerate(rects):
        new_area = 0
        left, top, right, bottom = rect
        for i in range(top, bottom):
            for j in range(left, right):
                if cv2.pointPolygonTest(contours[index], (j, i), False) >= 0 and new_mask[i, j] == 1:
                    new_area += 1
                    
        if new_area <= areas[index] * delete_threshold:
            for i in range(top, bottom):
                for j in range(left, right):
                    if cv2.pointPolygonTest(contours[index], (j, i), False) >= 0:
                        mask[i, j] = 0
        
        elif new_area <= areas[index] * replace_threshold:
            for i in range(top, bottom):
                for j in range(left, right):
                    if cv2.pointPolygonTest(contours[index], (j, i), False) >= 0:
                        mask[i, j] = new_mask[i, j]
    return mask

In [16]:
new_label = mask_replace_non_exist(pre_label.reshape(3000, 15106, 1), Ensemble.reshape(3000, 15106, 1), new_rects, new_contours, new_areas, replace_threshold=0.99, delete_threshold=0.05)

In [17]:
new_label = new_label.reshape(3000, 15106, 1)
new_label.shape, new_label.dtype

((3000, 15106, 1), dtype('uint8'))

In [18]:
cv2.imwrite('rejudge.bmp', new_label*255)

True

In [21]:
tiff.imsave('rejudge.tif', new_label)